In [1]:
import torch
import numpy as np
import glob
import torch.nn.functional as F
import soundfile as sf
from IPython.display import Audio
torch.set_num_threads(1)
from utils import init_jit_model, STFTExtractor, get_speech_ts, read_audio, state_generator
extractor = STFTExtractor()

/opt/conda/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchaudio/backend/utils.py:63: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to "soundfile". Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


## Full audio example

In [14]:
def collect_speeches(tss, wav):
    speech_chunks = []
    for i in tss:
        speech_chunks.append(wav[i['start']: i['end']])
    return np.concatenate(speech_chunks)

In [10]:
model = init_jit_model('files/joint_VAD_just_RU_jit_cut_q.pth.tar', 'cpu')

In [16]:
wav = read_audio('files/test_audio_6.wav')

In [17]:
speech_timestamps = get_speech_ts(wav, model, extractor, num_steps=4)

In [18]:
sf.write('only_speech.wav', collect_speeches(speech_timestamps, wav), 16000)
Audio('only_speech.wav')

## Stream example

In [7]:
model = init_jit_model('files/joint_VAD_just_RU_jit_cut_q.pth.tar', 'cpu')

In [8]:
audios_for_stream = glob.glob('files/test*.wav')
len(audios_for_stream)

10

In [9]:
for i in state_generator(model, audios_for_stream, extractor, audios_in_stream=2):
    if i:
        print(i)

Done initial Loading
[({106500: 'start'}, 'files/test_audio_1.wav')]
[({174000: 'start'}, 'files/test_audio_3.wav')]
[({261000: 'end'}, 'files/test_audio_1.wav')]
Loading next wav:  files/test_audio_7.wav
[({134000: 'start'}, 'files/test_audio_7.wav')]
[({147500: 'end'}, 'files/test_audio_7.wav')]
[({442000: 'end'}, 'files/test_audio_3.wav')]
[({450500: 'start'}, 'files/test_audio_3.wav')]
[({209500: 'start'}, 'files/test_audio_7.wav')]
[({519500: 'end'}, 'files/test_audio_3.wav')]
[({533500: 'start'}, 'files/test_audio_3.wav')]
[({599904: 'end'}, 'files/test_audio_3.wav')]
Loading next wav:  files/test_audio_6.wav
[({183500: 'start'}, 'files/test_audio_6.wav')]
[({503500: 'end'}, 'files/test_audio_7.wav')]
[({202500: 'end'}, 'files/test_audio_6.wav')]
[({537500: 'start'}, 'files/test_audio_7.wav')]
[({226500: 'start'}, 'files/test_audio_6.wav')]
[({283500: 'end'}, 'files/test_audio_6.wav')]
[({616500: 'end'}, 'files/test_audio_7.wav')]
[({337500: 'start'}, 'files/test_audio_6.wav')]
[